In [1]:
# import libraries (non-cryptogaphic)
import random # to generate phone numbers
import pandas as pd

# import libraries (cryptographic)
import cryptography.hazmat.primitives.asymmetric.dh as dh
from cryptography.fernet import Fernet
import hashlib
import sympy
import secrets

# Generate phone numbers and store them in phone_numbers.csv file

In [2]:
random.seed(10) # to ensure same phone numbers generated every time

In [3]:
# class to generate phone numbers for grab and gojek
class PhoneNumberGenerator:
    def __call__(self, count):
        phone_numbers = random.sample(range(80000000,100000000), count)
        return phone_numbers 

class FakePhoneNumberGenerator:
    def __call__(self, count):
        numbers = random.sample(range(10000000,80000000), count) # invalid phone numbers generated
        return numbers
            
    
# class to store numbers to csv
class PhoneNumberStorageManager:
    def __init__(self):
        self.filename = "phone_numbers_padded.csv"
    def __call__(self, gojek_phone_numbers, grab_phone_numbers):
        d = {"gojek": gojek_phone_numbers, 
            "grab": grab_phone_numbers}
        df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()])) # create dataframe
        df.to_csv(self.filename, index = False) # store values to file "phone_numbers_padded.csv"

In [4]:
set_size = 100 # both parties are to exchange sets of a pre-determined size

# generate and store phone numbers
gojek_phone_number_count = 61 # inclusive of phone numbers in common with grab
grab_phone_number_count = 91 # inclusive of phone numbers in common with gojek
common_phone_number_count = 10

# instantiate required classes
phone_number_generator = PhoneNumberGenerator()
fake_phone_number_generator = FakePhoneNumberGenerator()
phone_number_storage_manager = PhoneNumberStorageManager()

# generate phone numbers
phone_numbers = phone_number_generator(gojek_phone_number_count+grab_phone_number_count-common_phone_number_count)
common_phone_numbers = phone_numbers[0:common_phone_number_count]
gojek_phone_numbers = phone_numbers[0:gojek_phone_number_count]
grab_phone_numbers = common_phone_numbers + phone_numbers[gojek_phone_number_count:] 

# generate fake phone numbers
gojek_fake_phone_numbers = fake_phone_number_generator(set_size-gojek_phone_number_count)
grab_fake_phone_numbers = fake_phone_number_generator(set_size-grab_phone_number_count)

# add fake phone numbers to phone numbers
gojek_padded_set = gojek_phone_numbers + gojek_fake_phone_numbers
grab_padded_set = grab_phone_numbers + grab_fake_phone_numbers

# shuffle phone number lists
random.shuffle(gojek_padded_set)
random.shuffle(grab_padded_set)

# write phone numbers (with fake ones) to csv file
phone_number_storage_manager(gojek_padded_set, grab_padded_set)

# Define classes for the necessary for the algorithm

In [5]:
# class to generate numbers required for psi
class NumberGenerator:
    
    def generate_public_parameters(self, size):
        # method to generate p, q and factors of p-1
        p = self.generate_safe_prime(size)
        print(f"p is prime: {sympy.ntheory.isprime(p)}")
        length_of_p = len(bin(p)[2:]) # should be 1024
        print(f"Length of prime modulus, p: {length_of_p}.\nNote: Should be {size}.")
        q = (p-1)//2
        print(f"q is prime: {sympy.ntheory.isprime(q)}")
        factors_pminus1 = [1, 2, q] # since q is prime, 2q only has these 3 factors excluding itself
                                    # 2q = p-1
                                    # factors of p-1 required to compute order of generators (lagrange theorem)
        return p, factors_pminus1

    def generate_safe_prime(self, size):
        # method to generate safe prime for p
        candidate = dh.generate_parameters(2, size).parameter_numbers().p # generate 1024-bit prime number
        while True:
            # the method used from the cryptography already generates a safe prime, this portion is merely to double confirm
            is_safe_prime = sympy.ntheory.isprime((candidate-1)//2) # if safe prime, (candidate-1)/2 is prime
            if (is_safe_prime):
                break
            else:
                candidate = dh.generate_parameters(2, size).parameter_numbers().p
                print(candidate)
 
        return candidate
    
    
    def generate_random_number(self, size):
        # method to generate client's secret
        return secrets.randbits(size)
            
# class to inspect values          
class NumberInspector:
    
    def check_is_primitive_generator(self, candidate, factors_divisorminus1, divisor): # note: factors should be the factors of divisor-1
        
        # apply lagrange theorem
        for possible_order in factors_divisorminus1: # check congruence for all factors (factors is exclusive of p-1 itself)
            result = pow(candidate, possible_order, divisor) # fast modular exponentiation implemented in pow(x,y,z)
                                                             # python integers have arbitrary precisions, 
                                                             # no overflow would occur if operations done in pure python
            if (result == 1):
                return False # if candidate exponent any of the factors (1,2,q) congruent to 1modp, candidate is not a primitive
                             # generator
        return True # order of candidate == p-1, therefore candidate is a primitive generator

    
class StorageManager:
    
    def store_data(self, filename, data):
        df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in data.items()])) # create dataframe
        df.to_csv(filename, index=False) # store data to file with filename


# Create psi client class

In [6]:
# client class (both grab and gojek are clients communicating directly with each other)
class Client:
    def __init__(self, name, other_party_name, private_key_size, phone_numbers, p, factors_pminus1, fernet_key):
        
        self.number_inspector = NumberInspector()
        self.number_generator = NumberGenerator()
        
        # for asymmetric encryption
        self.private_key = self.number_generator.generate_random_number(private_key_size)
        self.my_set = phone_numbers
        self.p = p # prime modulus
        self.factors_pminus1 = factors_pminus1 # to calculate order of hashed phone numbers, 
                                               # since algorithm requires them to be primitive generators
        
        # for symmetric encryption
        self.fernet_key = fernet_key
        self.f = Fernet(fernet_key)
        
        # values to track for psi algorithm
        self.my_hashed_set = None # h(x)
        self.my_self_encrypted_set = None # (h(x)^(my_secret))modp
        self.my_encrypted_set = None # (h(x)^(my_secret)(other_party_secret))modp
        self.other_party_encrypted_set = None # (h(y)^(my_secret)(other_party_secret))modp
        self.common_values = None # common phone numbers
        
        
        # create datafile for communication with another party
        
        # content to store in file
        # only need to share self_encrypted_values and other_party_encrypted_values
        # common_values to ensure both calculate the same intersection
        self.my_dict = {
            'my_self_encrypted_set': None, 
            'other_party_encrypted_set': None,
            'common_values': None
        }
        
        # filenames
        self.name = name
        self.filename = name + "_data_v1.2.csv"
        self.other_party_name = other_party_name
        self.other_party_filename = other_party_name + "_data_v1.2.csv"
        
        # create file
        self.storage_manager = StorageManager()
        self.storage_manager.store_data(self.filename, self.my_dict)

    def hash_to_primitive_root_modulo_p(self, element): 
        # method to hash phone numbers to primitive root modulo p i.e. primitive generator

        endian = "big"
        element = element.to_bytes(4, endian)
        hash_hex = hashlib.sha256(element).hexdigest() # sha3_256
        hash_int = int(hash_hex, 16)
        while True:
            # repeatedly hash until primitive root modulo p is obtained
            is_primitive_generator = self.number_inspector.check_is_primitive_generator(
                hash_int, self.factors_pminus1, self.p
            )
            if (is_primitive_generator):
                break
            else:
                hash_int = hash_int.to_bytes(32, endian)
                hash_hex = hashlib.sha256(hash_int).hexdigest()
                hash_int = int(hash_hex, 16)
                
        return hash_int
    
    def modular_exponentation(self, element):
        # compute (element^(private_key))modp
    
        return pow(element, self.private_key, self.p)
    
    def hash_set(self):
        # hash all phone numbers in my set to primitive root modulo p, one by one
        
        self.my_hashed_set = []
        
        for element in self.my_set:
            hashed_value = self.hash_to_primitive_root_modulo_p(element)
            self.my_hashed_set.append(hashed_value)
            
    def encrypt_set(self, is_other_party):
        # encrypt all elements in a given set using private_key, one by one
        
        # two scenarios to consider
        # one: encrypt set sent by the other party
        if (is_other_party):
            decrypted_other_party_set = self.receive_data("my_self_encrypted_set")
            other_party_set_int = []
            for element_string in decrypted_other_party_set:
                other_party_set_int.append(int(element_string))
            set_to_encrypt = other_party_set_int
        # two: encrypt my own set
        else:
            set_to_encrypt = self.my_hashed_set
        
        # encrypt values in given set, one by one
        encrypted_values = []
        for element in set_to_encrypt:
            encrypted_value = self.modular_exponentation(element)
            encrypted_values.append(encrypted_value)
            
        # assign the encrypted set to the correct variable
        # update csv file used for communication
        if (is_other_party):
            self.other_party_encrypted_set = encrypted_values
            self.send_data(encrypted_values, "other_party_encrypted_set")
        else:
            self.my_self_encrypted_set = encrypted_values
            self.send_data(encrypted_values, "my_self_encrypted_set")
                    
        
    def get_intersection(self):
        
        # get intersection
        
        my_encrypted_set = self.receive_data("other_party_encrypted_set") # read my encrypted set from the other party's file
        my_encrypted_set_int = []
        
        # convert read values to integer
        for element in my_encrypted_set:
            my_encrypted_set_int.append(int(element))
            
        # assign to correct variable
        self.my_encrypted_set = my_encrypted_set_int
        
        # get intersection
        encrypted_common_values = set(self.my_encrypted_set).intersection(self.other_party_encrypted_set)
        index_of_common_values = []
        
        # find the index of the elements in the intersection in my_encrypted_set
        for element in encrypted_common_values:
            index_of_common_values.append(self.my_encrypted_set.index(element))
            
        self.common_values = []
        
        # find the values in my own set corresponding to the index of the elements in the intersection
        for index in index_of_common_values:
            potential_phone_number = self.my_set[index]
            
            if ((potential_phone_number-79999999)>0): # remove any potential fake numbers that intersect
                self.common_values.append(potential_phone_number)
        
        # update csv file for communication
        self.send_data(self.common_values, "common_values")
    
    def encrypt_data(self, plaintext):
        # encrypt data with Fernet
        
        endian = "big"
        element = plaintext.to_bytes(128, endian) # 1024 bits == 128 bytes
        cipher_text = self.f.encrypt(element)
        return cipher_text
    
    def decrypt_data(self, ciphertext):
        # decrypt data encrypted by Fernet
        
        ciphertext_bytes = ciphertext.encode('utf-8')[2:-1] # convert from string back to bytes
        endian = "big"
        element_in_bytes = self.f.decrypt(ciphertext_bytes)
        plaintext = int.from_bytes(element_in_bytes, endian)
        return plaintext

    def send_data(self, data_to_send, column_name):
        # send data means writing to file. encrypt data with Fernet
        
        # encrypt data
        encrypted_data_to_send = []
        for element in data_to_send:
            encrypted_element = self.encrypt_data(element)
            encrypted_data_to_send.append(encrypted_element)
            
        # send data
        self.my_dict[column_name] = encrypted_data_to_send
        self.storage_manager.store_data(self.filename, self.my_dict)
        
        
    def receive_data(self, column_name):
        # receive data means reading from file (my file). decrypt data encrypted by Fernet
        
        # receive data
        encrypted_data = self.get_other_party_data()[column_name].to_list()
        
        # decrypt data
        decrypted_data = []
        for element in encrypted_data:
            if type(element) is float: # remove NaN
                continue
            decrypted_element = self.decrypt_data(element)
            decrypted_data.append(decrypted_element)
        print(f"number of elements in {self.other_party_name};{column_name}: {len(decrypted_data)}.")
        return decrypted_data
    
    def get_my_data(self):
        # read my file as dataframe (other party's file)
        
        return pd.read_csv(self.filename)
            
    def get_other_party_data(self):
        # read other party's file as dataframe
        
        return pd.read_csv(self.other_party_filename)
    
    


# Initialize context

In [7]:
# assign pre-determined variables for psi
key_size = 1024 # both private keys and large prime

# create key for symmetric key cryptography
fernet_key = Fernet.generate_key()

# create public parameters required for psi
number_generator = NumberGenerator()
p, factors_pminus1 = number_generator.generate_public_parameters(key_size)

# create clients
grab = Client("grab", "gojek", key_size, grab_padded_set, p, factors_pminus1, fernet_key)
gojek = Client("gojek", "grab", key_size, gojek_padded_set, p, factors_pminus1, fernet_key)



p is prime: True
Length of prime modulus, p: 1024.
Note: Should be 1024.
q is prime: True


C:\Users\jiaji\AppData\Local\Temp/ipykernel_13828/2734410534.py:55: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in data.items()])) # create dataframe


# Get intersection

## Step 1: Hash phone numbers

In [8]:
# clients hash their own set
grab.hash_set()
gojek.hash_set()

### Clients' status after step 1
Note: hashed set is not stored in the file as it is not meant to be shared with the other party hence, files' status after step 1 is not shown

In [9]:
d = {"gojek hashed set": gojek.my_hashed_set,
    "grab hashed set": grab.my_hashed_set}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()]))
df

,gojek hashed set,grab hashed set
0,3444554169003369415400187425444370377909224933...,4440918517132468010599612783354082194985937761...
1,3134205932310820582994345236779128274482638005...,5706214998945741993120430770407017796268789491...
2,8972686745761283250343305795976962558216403891...,8119100922771943066177300284568368128847410924...
3,5203358287300148779441149003132562463607163612...,2835741223782226842638506406959946569416451500...
4,8810634112203516697597708414004325272706740649...,4320682999770379726688099447684191745405506617...
...,...,...
95,8697318885625964372170362596957150767783631535...,2064846653753480132804612906272453669387169259...
96,1516234710201293820270438455976630785218809049...,8866985291088494051041055141151271041321507008...
97,1050996311389914722852387111588332931098320412...,8991368697896120309677236639872622257987441175...
98,9565788091314602955585247733130100648913305029...,7018556158959834041117086565402060628499966280...


## Step 2: encrypt hashed set with own private key

In [10]:
# clients self encrypt hashed set
grab.encrypt_set(False) # set is_other_party to false to encrypt own hashed set
gojek.encrypt_set(False)

C:\Users\jiaji\AppData\Local\Temp/ipykernel_13828/2734410534.py:55: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in data.items()])) # create dataframe


### Clients' status after step 2 (value of variables in client)

In [11]:
d = {"gojek hashed set": gojek.my_hashed_set,
    "grab hashed set": grab.my_hashed_set,
    "gojek self-encrypted set": gojek.my_self_encrypted_set,
    "grab self-encrypted set": grab.my_self_encrypted_set}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()]))
df

,gojek hashed set,grab hashed set,gojek self-encrypted set,grab self-encrypted set
0,3444554169003369415400187425444370377909224933...,4440918517132468010599612783354082194985937761...,6965353149748059875212735538008504568833444088...,1061045942032031254497721381533980568579536223...
1,3134205932310820582994345236779128274482638005...,5706214998945741993120430770407017796268789491...,4210046765084092963672629919412148359542301858...,9012385047708532751340985482142624731781133783...
2,8972686745761283250343305795976962558216403891...,8119100922771943066177300284568368128847410924...,7979022908767925879134094727370120289668897036...,7228204620079225016085593088283948911575116211...
3,5203358287300148779441149003132562463607163612...,2835741223782226842638506406959946569416451500...,6743910644870155550868460131705030089775847423...,6438675757646013610405476001490934756539950223...
4,8810634112203516697597708414004325272706740649...,4320682999770379726688099447684191745405506617...,8962619428706799707003773092691447455818131932...,1035329197465001675551896636417557181096287162...
...,...,...,...,...
95,8697318885625964372170362596957150767783631535...,2064846653753480132804612906272453669387169259...,3850640924695316466797596305063484710576947109...,7742872220191479853792345906982859359978734546...
96,1516234710201293820270438455976630785218809049...,8866985291088494051041055141151271041321507008...,5108700378844592453367446554000523474679617036...,9426454674150398352293447530576061392311108419...
97,1050996311389914722852387111588332931098320412...,8991368697896120309677236639872622257987441175...,6812067013058853416650544402776485879431760838...,3420652382761238616333717218727918503334192319...
98,9565788091314602955585247733130100648913305029...,7018556158959834041117086565402060628499966280...,6047304055341729576233039417475053732305988868...,7427656167156385550289488990394159515029507997...


### Files' status after step 2 (value of variables in file - clients' variables encrypted with Fernet)

In [12]:
df_gojek = gojek.get_my_data()
df_grab = grab.get_my_data()
print("gojek's file:")
df_gojek


gojek's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,b'gAAAAABhogPX1lBcOou5a71DvN1HwNF1tndxItAXq4lw...,NaN,NaN
1,b'gAAAAABhogPXFqmP86ud-Ru44DRF-khwjynSybL0Jmgx...,NaN,NaN
2,b'gAAAAABhogPXXICP46rMydT6Mxhy_FbFBGL9TCFB0ZYj...,NaN,NaN
3,b'gAAAAABhogPXHw2kApcSU7zJKqUV5yY6sssCG0gX0nhu...,NaN,NaN
4,b'gAAAAABhogPXEIXH6HboMocZExOAcrlrmNIFD8ekNuOb...,NaN,NaN
...,...,...,...
95,b'gAAAAABhogPXSIWlNUW_DmOQ8ZEgYU2W1hU96QXiRWAf...,NaN,NaN
96,b'gAAAAABhogPXzA_jpWH8RTCnGMwfp4Qhv1y_rBJGWKN2...,NaN,NaN
97,b'gAAAAABhogPXoc7W8SsPn67vkdCMt04O2fhcz3qSQHZ7...,NaN,NaN
98,b'gAAAAABhogPXlSBYNvRO0usypQ1_kWa0Adza8xA86jIC...,NaN,NaN


In [13]:
print("grab's file:")
df_grab

grab's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,b'gAAAAABhogPXWbvAXZMbT6lGQ4sD1U627HAHLHPoQwau...,NaN,NaN
1,b'gAAAAABhogPXBpDDWFNxGmoUDEje7GPeZXCi4Xs1qsjM...,NaN,NaN
2,b'gAAAAABhogPX54pCYDy5aVYJyiFrkK5XxD3RxM5Py17a...,NaN,NaN
3,b'gAAAAABhogPXjGoUxyrXlSBpq6DoAQk6B-9mDM2PcvHN...,NaN,NaN
4,b'gAAAAABhogPXTq3bM_l0v0Pf-WXLB87XAwoUymavsDsa...,NaN,NaN
...,...,...,...
95,b'gAAAAABhogPXnZI469G34pykXoaSAphO1NSATGSMU2XR...,NaN,NaN
96,b'gAAAAABhogPXz4irE6r6FxuVEulK1iyN2ZrUNiCz7-zX...,NaN,NaN
97,b'gAAAAABhogPX3eaTVDUn5KQ8it_2QL0jyxaskYu6CGgC...,NaN,NaN
98,b'gAAAAABhogPXx7NofbuIjqQh1e8clC8imiU1BrDwEz95...,NaN,NaN


## Step 3: encrypt other party's self-encrypted set with own private key


In [14]:
# clients encrypt other party's self encrypted set
grab.encrypt_set(True) # set is_other_party to true
gojek.encrypt_set(True)

number of elements in gojek;my_self_encrypted_set: 100.


C:\Users\jiaji\AppData\Local\Temp/ipykernel_13828/2734410534.py:55: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in data.items()])) # create dataframe


number of elements in grab;my_self_encrypted_set: 100.


### Clients' status after step 3

In [15]:
d = {"gojek hashed set": gojek.my_hashed_set,
    "grab hashed set": grab.my_hashed_set,
    "gojek self-encrypted set": gojek.my_self_encrypted_set,
    "grab self-encrypted set": grab.my_self_encrypted_set,
    "gojek encrypted set": grab.other_party_encrypted_set,
    "grab encrypted set": gojek.other_party_encrypted_set}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()]))
df

,gojek hashed set,grab hashed set,gojek self-encrypted set,grab self-encrypted set,gojek encrypted set,grab encrypted set
0,3444554169003369415400187425444370377909224933...,4440918517132468010599612783354082194985937761...,6965353149748059875212735538008504568833444088...,1061045942032031254497721381533980568579536223...,1044765099765992877315510120280676086995977889...,6783780555455013316172022292625804619453559677...
1,3134205932310820582994345236779128274482638005...,5706214998945741993120430770407017796268789491...,4210046765084092963672629919412148359542301858...,9012385047708532751340985482142624731781133783...,9989809815151636143939847524772477936180544073...,4135287716107478446307247773894485113108163305...
2,8972686745761283250343305795976962558216403891...,8119100922771943066177300284568368128847410924...,7979022908767925879134094727370120289668897036...,7228204620079225016085593088283948911575116211...,8382500027729880760797173113439223470047228187...,7514057904332171139937315632706401385844890440...
3,5203358287300148779441149003132562463607163612...,2835741223782226842638506406959946569416451500...,6743910644870155550868460131705030089775847423...,6438675757646013610405476001490934756539950223...,1810406201841679057151155716223540224701126727...,8828704629380523187981253613320991992072265080...
4,8810634112203516697597708414004325272706740649...,4320682999770379726688099447684191745405506617...,8962619428706799707003773092691447455818131932...,1035329197465001675551896636417557181096287162...,7390985736681681023004953347696067214819003245...,1298917437312396261398855741682596930745050609...
...,...,...,...,...,...,...
95,8697318885625964372170362596957150767783631535...,2064846653753480132804612906272453669387169259...,3850640924695316466797596305063484710576947109...,7742872220191479853792345906982859359978734546...,9400439090608088599177018588329685424659541558...,1377649574966961550293284493847777577453761116...
96,1516234710201293820270438455976630785218809049...,8866985291088494051041055141151271041321507008...,5108700378844592453367446554000523474679617036...,9426454674150398352293447530576061392311108419...,2352584793045652322561948343724447556652410929...,8025747950895767225888597600981781370964139476...
97,1050996311389914722852387111588332931098320412...,8991368697896120309677236639872622257987441175...,6812067013058853416650544402776485879431760838...,3420652382761238616333717218727918503334192319...,6717823515783158029786682509054484559003103476...,8617100046316004361344228830605289396537315732...
98,9565788091314602955585247733130100648913305029...,7018556158959834041117086565402060628499966280...,6047304055341729576233039417475053732305988868...,7427656167156385550289488990394159515029507997...,4134489884100283726613455702539612052407529853...,3709118941033501635357552460815312064494145336...


### Files' status after step 3

In [16]:
df_gojek = gojek.get_my_data()
df_grab = grab.get_my_data()
print("gojek's file:")
df_gojek



gojek's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,b'gAAAAABhogPX1lBcOou5a71DvN1HwNF1tndxItAXq4lw...,b'gAAAAABhogPY5usf4VBnLzwJRxyHztXzHNIcHzAFW5Jx...,NaN
1,b'gAAAAABhogPXFqmP86ud-Ru44DRF-khwjynSybL0Jmgx...,b'gAAAAABhogPYh1wSPezhRmaFd0s4V07piGdX3Cdl9zJD...,NaN
2,b'gAAAAABhogPXXICP46rMydT6Mxhy_FbFBGL9TCFB0ZYj...,b'gAAAAABhogPY3qfW3X_lLw6DczKxcPQOBqKa9PP71pEF...,NaN
3,b'gAAAAABhogPXHw2kApcSU7zJKqUV5yY6sssCG0gX0nhu...,b'gAAAAABhogPYhmLSkwz_-bNe3pO_Td7zc_0bOTkCLWKR...,NaN
4,b'gAAAAABhogPXEIXH6HboMocZExOAcrlrmNIFD8ekNuOb...,b'gAAAAABhogPY3y5KypHuip5ugsLrRtoDXyXmUlO0JwkA...,NaN
...,...,...,...
95,b'gAAAAABhogPXSIWlNUW_DmOQ8ZEgYU2W1hU96QXiRWAf...,b'gAAAAABhogPYqtYKnThSgD1IjFSei3nz7GoAQEUz71xx...,NaN
96,b'gAAAAABhogPXzA_jpWH8RTCnGMwfp4Qhv1y_rBJGWKN2...,b'gAAAAABhogPYTEgyu4yNN4OUia7tME6IW4h_DJFPI3KV...,NaN
97,b'gAAAAABhogPXoc7W8SsPn67vkdCMt04O2fhcz3qSQHZ7...,b'gAAAAABhogPYJLJWEEOIgkgYCZ0N6FeEh-BqaD5z1dM0...,NaN
98,b'gAAAAABhogPXlSBYNvRO0usypQ1_kWa0Adza8xA86jIC...,b'gAAAAABhogPY1E5GSsQH00AdATFHNaW_EuOZCTwsyeyU...,NaN


In [17]:
print("grab's file:")
df_grab

grab's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,b'gAAAAABhogPXWbvAXZMbT6lGQ4sD1U627HAHLHPoQwau...,b'gAAAAABhogPYeOu2rx3W5FtRcCB_Hb2wokmhr7CtPcMF...,NaN
1,b'gAAAAABhogPXBpDDWFNxGmoUDEje7GPeZXCi4Xs1qsjM...,b'gAAAAABhogPYoMUlUg038D8p9F5TJ7GjfTkBWR1YjLjA...,NaN
2,b'gAAAAABhogPX54pCYDy5aVYJyiFrkK5XxD3RxM5Py17a...,b'gAAAAABhogPYsVdkl7wMfuVa_BkLfl4-JlGDXuhGDpLq...,NaN
3,b'gAAAAABhogPXjGoUxyrXlSBpq6DoAQk6B-9mDM2PcvHN...,b'gAAAAABhogPY9Fmrv6d8TJGKld33nYrSt8U2s5vm7dam...,NaN
4,b'gAAAAABhogPXTq3bM_l0v0Pf-WXLB87XAwoUymavsDsa...,b'gAAAAABhogPYjCwrN0VqdoivGNRj6jbLPhUcOQHJb2RZ...,NaN
...,...,...,...
95,b'gAAAAABhogPXnZI469G34pykXoaSAphO1NSATGSMU2XR...,b'gAAAAABhogPYhQtZnDbkf6fLAYJNW8l4kNll7VOcc_Kb...,NaN
96,b'gAAAAABhogPXz4irE6r6FxuVEulK1iyN2ZrUNiCz7-zX...,b'gAAAAABhogPYgGlbZ3MmpOD7xUeI0Opl3AQuvwY1Zasb...,NaN
97,b'gAAAAABhogPX3eaTVDUn5KQ8it_2QL0jyxaskYu6CGgC...,b'gAAAAABhogPYgPdUV75bUquk0H60BOBL0YLcEoXPmmyP...,NaN
98,b'gAAAAABhogPXx7NofbuIjqQh1e8clC8imiU1BrDwEz95...,b'gAAAAABhogPYSgBaEzn2EnGgs8_hmq1DS_qHF8x_AZPg...,NaN


## Step 4: find intersection


In [18]:
# clients find intersection
grab.get_intersection()
gojek.get_intersection()

number of elements in gojek;other_party_encrypted_set: 100.
number of elements in grab;other_party_encrypted_set: 100.


### Clients' status after step 4:

In [19]:
d = {"gojek hashed set": gojek.my_hashed_set,
    "grab hashed set": grab.my_hashed_set,
    "gojek self-encrypted set": gojek.my_self_encrypted_set,
    "grab self-encrypted set": grab.my_self_encrypted_set,
    "gojek encrypted set": grab.other_party_encrypted_set,
    "grab encrypted set": gojek.other_party_encrypted_set,
    "gojek found intersection": gojek.common_values,
    "grab found intersection:": grab.common_values}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()]))
df

,gojek hashed set,grab hashed set,gojek self-encrypted set,grab self-encrypted set,gojek encrypted set,grab encrypted set,gojek found intersection,grab found intersection:
0,3444554169003369415400187425444370377909224933...,4440918517132468010599612783354082194985937761...,6965353149748059875212735538008504568833444088...,1061045942032031254497721381533980568579536223...,1044765099765992877315510120280676086995977889...,6783780555455013316172022292625804619453559677...,96192082.0,96192082.0
1,3134205932310820582994345236779128274482638005...,5706214998945741993120430770407017796268789491...,4210046765084092963672629919412148359542301858...,9012385047708532751340985482142624731781133783...,9989809815151636143939847524772477936180544073...,4135287716107478446307247773894485113108163305...,80497694.0,80497694.0
2,8972686745761283250343305795976962558216403891...,8119100922771943066177300284568368128847410924...,7979022908767925879134094727370120289668897036...,7228204620079225016085593088283948911575116211...,8382500027729880760797173113439223470047228187...,7514057904332171139937315632706401385844890440...,81093373.0,81093373.0
3,5203358287300148779441149003132562463607163612...,2835741223782226842638506406959946569416451500...,6743910644870155550868460131705030089775847423...,6438675757646013610405476001490934756539950223...,1810406201841679057151155716223540224701126727...,8828704629380523187981253613320991992072265080...,94391128.0,94391128.0
4,8810634112203516697597708414004325272706740649...,4320682999770379726688099447684191745405506617...,8962619428706799707003773092691447455818131932...,1035329197465001675551896636417557181096287162...,7390985736681681023004953347696067214819003245...,1298917437312396261398855741682596930745050609...,86915509.0,86915509.0
...,...,...,...,...,...,...,...,...
95,8697318885625964372170362596957150767783631535...,2064846653753480132804612906272453669387169259...,3850640924695316466797596305063484710576947109...,7742872220191479853792345906982859359978734546...,9400439090608088599177018588329685424659541558...,1377649574966961550293284493847777577453761116...,NaN,NaN
96,1516234710201293820270438455976630785218809049...,8866985291088494051041055141151271041321507008...,5108700378844592453367446554000523474679617036...,9426454674150398352293447530576061392311108419...,2352584793045652322561948343724447556652410929...,8025747950895767225888597600981781370964139476...,NaN,NaN
97,1050996311389914722852387111588332931098320412...,8991368697896120309677236639872622257987441175...,6812067013058853416650544402776485879431760838...,3420652382761238616333717218727918503334192319...,6717823515783158029786682509054484559003103476...,8617100046316004361344228830605289396537315732...,NaN,NaN
98,9565788091314602955585247733130100648913305029...,7018556158959834041117086565402060628499966280...,6047304055341729576233039417475053732305988868...,7427656167156385550289488990394159515029507997...,4134489884100283726613455702539612052407529853...,3709118941033501635357552460815312064494145336...,NaN,NaN


Note: Last 2 columns, unlike the rest of the columns, do not have a one-to-one mapping with other values belonging to the same row i.e. values in the last 2 columns do not have any relation to the other values in the same row as it.

### Files' status after step 4

In [20]:
df_gojek = gojek.get_my_data()
df_grab = grab.get_my_data()
print("gojek's file:")
df_gojek


gojek's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,b'gAAAAABhogPX1lBcOou5a71DvN1HwNF1tndxItAXq4lw...,b'gAAAAABhogPY5usf4VBnLzwJRxyHztXzHNIcHzAFW5Jx...,b'gAAAAABhogPZfx0ZaNcSHaPGbxEYseMrC2-nz6SnZHSX...
1,b'gAAAAABhogPXFqmP86ud-Ru44DRF-khwjynSybL0Jmgx...,b'gAAAAABhogPYh1wSPezhRmaFd0s4V07piGdX3Cdl9zJD...,b'gAAAAABhogPZ1U64GWBC246Vj2jqjVhkFuAqTqGjn0J8...
2,b'gAAAAABhogPXXICP46rMydT6Mxhy_FbFBGL9TCFB0ZYj...,b'gAAAAABhogPY3qfW3X_lLw6DczKxcPQOBqKa9PP71pEF...,b'gAAAAABhogPZfpLNsDvPrjTuxqnnpYTUA2hMKoZPWnBM...
3,b'gAAAAABhogPXHw2kApcSU7zJKqUV5yY6sssCG0gX0nhu...,b'gAAAAABhogPYhmLSkwz_-bNe3pO_Td7zc_0bOTkCLWKR...,b'gAAAAABhogPZB8dWtSPoadmy9cIFL7EQTf_Cw0fV9TUf...
4,b'gAAAAABhogPXEIXH6HboMocZExOAcrlrmNIFD8ekNuOb...,b'gAAAAABhogPY3y5KypHuip5ugsLrRtoDXyXmUlO0JwkA...,b'gAAAAABhogPZRkWzWYDZOr0BCjLlT8VRKgKVHU_xmX4Q...
...,...,...,...
95,b'gAAAAABhogPXSIWlNUW_DmOQ8ZEgYU2W1hU96QXiRWAf...,b'gAAAAABhogPYqtYKnThSgD1IjFSei3nz7GoAQEUz71xx...,NaN
96,b'gAAAAABhogPXzA_jpWH8RTCnGMwfp4Qhv1y_rBJGWKN2...,b'gAAAAABhogPYTEgyu4yNN4OUia7tME6IW4h_DJFPI3KV...,NaN
97,b'gAAAAABhogPXoc7W8SsPn67vkdCMt04O2fhcz3qSQHZ7...,b'gAAAAABhogPYJLJWEEOIgkgYCZ0N6FeEh-BqaD5z1dM0...,NaN
98,b'gAAAAABhogPXlSBYNvRO0usypQ1_kWa0Adza8xA86jIC...,b'gAAAAABhogPY1E5GSsQH00AdATFHNaW_EuOZCTwsyeyU...,NaN


In [21]:
print("grab's file:")
df_grab

grab's file:


,my_self_encrypted_set,other_party_encrypted_set,common_values
0,b'gAAAAABhogPXWbvAXZMbT6lGQ4sD1U627HAHLHPoQwau...,b'gAAAAABhogPYeOu2rx3W5FtRcCB_Hb2wokmhr7CtPcMF...,b'gAAAAABhogPZ5x_r15x2piXryayXculHLWYwys5ZGO8r...
1,b'gAAAAABhogPXBpDDWFNxGmoUDEje7GPeZXCi4Xs1qsjM...,b'gAAAAABhogPYoMUlUg038D8p9F5TJ7GjfTkBWR1YjLjA...,b'gAAAAABhogPZy-MCq5ADtMoBFcHYwgjPWrPUon3JazX4...
2,b'gAAAAABhogPX54pCYDy5aVYJyiFrkK5XxD3RxM5Py17a...,b'gAAAAABhogPYsVdkl7wMfuVa_BkLfl4-JlGDXuhGDpLq...,b'gAAAAABhogPZf1JVh7CW6HCfbkdgZBvFhXcojsYcvyL4...
3,b'gAAAAABhogPXjGoUxyrXlSBpq6DoAQk6B-9mDM2PcvHN...,b'gAAAAABhogPY9Fmrv6d8TJGKld33nYrSt8U2s5vm7dam...,b'gAAAAABhogPZrKybUS5UmDcyzwkACoDVtHgNhq_aJsxk...
4,b'gAAAAABhogPXTq3bM_l0v0Pf-WXLB87XAwoUymavsDsa...,b'gAAAAABhogPYjCwrN0VqdoivGNRj6jbLPhUcOQHJb2RZ...,b'gAAAAABhogPZigo1LJknqkwcX68dCNY-WwMuy0tPUM0X...
...,...,...,...
95,b'gAAAAABhogPXnZI469G34pykXoaSAphO1NSATGSMU2XR...,b'gAAAAABhogPYhQtZnDbkf6fLAYJNW8l4kNll7VOcc_Kb...,NaN
96,b'gAAAAABhogPXz4irE6r6FxuVEulK1iyN2ZrUNiCz7-zX...,b'gAAAAABhogPYgGlbZ3MmpOD7xUeI0Opl3AQuvwY1Zasb...,NaN
97,b'gAAAAABhogPX3eaTVDUn5KQ8it_2QL0jyxaskYu6CGgC...,b'gAAAAABhogPYgPdUV75bUquk0H60BOBL0YLcEoXPmmyP...,NaN
98,b'gAAAAABhogPXx7NofbuIjqQh1e8clC8imiU1BrDwEz95...,b'gAAAAABhogPYSgBaEzn2EnGgs8_hmq1DS_qHF8x_AZPg...,NaN


# Check results

In [22]:
# get intersection found by the two parties
gojek_found_intersection = gojek.common_values
grab_found_intersection = grab.common_values


# sort numbers for easier comparison
gojek_found_intersection.sort()
grab_found_intersection.sort()
common_phone_numbers.sort()

# summarize them in a dataframe
d = {"actual": common_phone_numbers,
    "gojek": gojek_found_intersection,
    "grab": grab_found_intersection}
df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in d.items()]))
df

,actual,gojek,grab
0,80497694,80497694,80497694
1,81093373,81093373,81093373
2,86915509,86915509,86915509
3,89312048,89312048,89312048
4,94391128,94391128,94391128
5,95521626,95521626,95521626
6,96192082,96192082,96192082
7,96485172,96485172,96485172
8,99173089,99173089,99173089
9,99397525,99397525,99397525
